In [ ]:
! pip install wandb
# grab @TheadoreGalanos's fork for config file
! git clone https://github.com/TheodoreGalanos/gpt-neox/
import wandb
import pandas as pd

In [ ]:
# initialize config
sweep_config = {
  "name": "Scaling laws sweep",
  "method": "grid",
  "parameters": {
    "valid_set": {
        "distribution": "categorical",
        'values': [
          ### This will be a list of strings
          ### with the following values:
          ### n_layer, n_head, n_embd, head_state
          ### separated by commas
      ]
    }
  }
}

# read TG's config
df = pd.read_csv(
    './gpt-neox/configs/scaling_experiment/config_parameters.csv',
    )

# create strings for each row
for i in range(len(df)):
  sweep_config['parameters']['valid_set']['values'].append(
      ','.join([str(df.iloc[i][x]) for x in df.columns])
  )


In [ ]:
# check source
print(df)

# and config
sweep_config

In [ ]:
# test run
sweep_id = wandb.sweep(sweep_config)

def train():
    run = wandb.init()
    print(run.config.valid_set)
    exponent, n_layer, n_head, n_embd, head_state = run.config.valid_set.split(',')
    print(exponent)
    run.finish()

sweep_id = wandb.sweep(sweep_config)
agent = wandb.agent(sweep_id=sweep_id, function=train)
agent.run()